In [1]:
!pip3 install apache_beam

In [2]:
import apache_beam as beam

## **CombineGlobally**:

*   Combines all elements in a collection.


The more general way to combine elements, and the most flexible, is with a class that inherits from CombineFn.

1. **CombineFn.create_accumulator():** This creates an empty accumulator. For example, an empty accumulator for a sum would be 0, while an empty accumulator for a product (multiplication) would be 1.

2. **CombineFn.add_input():** Called once per element. Takes an accumulator and an input element, combines them and returns the updated accumulator.

3. **CombineFn.merge_accumulators():** Multiple accumulators could be processed in parallel, so this function helps merging them into a single accumulator.

4. **CombineFn.extract_output():** It allows to do additional calculations before extracting a result.

In [3]:
class AverageFn(beam.CombineFn):
  def create_accumulator(self):
    return (0.0, 0)

  def add_input(self, sum_count, input):
    (sum, count) = sum_count
    return sum + input, count + 1

  def merge_accumulators(self, accumulators):
    sums, counts = zip(*accumulators)
    return sum(sums), sum(counts)

  def extract_output(self, sum_count):
    (sum, count) = sum_count
    return sum / count if count else float('NaN')

In [4]:
with beam.Pipeline() as p:
  input_data = (p
                | "Create data" >> beam.Create([21,45,78,99,1,22,5])
                | "Combine Globally" >> beam.CombineGlobally(AverageFn())
                |"Write to Local">> beam.io.WriteToText('data/result'))

In [5]:
!{'head -n 10 data/result-00000-of-00001'}

38.714285714285715


## **CombinePerKey:**

*   Combines all elements for each key in a collection.

In [6]:
with beam.Pipeline() as pipeline:
  total = (
      pipeline
      | 'Create plant counts' >> beam.Create([
          ('🥕', 3),
          ('🥕', 2),
          ('🍆', 1),
          ('🍅', 4),
          ('🍅', 5),
          ('🍅', 3),
      ])
      | 'Sum' >> beam.CombinePerKey(sum)
      | beam.Map(print))

('🥕', 5)
('🍆', 1)
('🍅', 12)


**We define a function saturated_sum which takes an iterable of numbers and adds them together, up to a predefined maximum number.**

In [7]:
def saturated_sum(values):
  max_value = 8
  return min(sum(values), max_value)

with beam.Pipeline() as pipeline:
  saturated_total = (
      pipeline
      | 'Create plant counts' >> beam.Create([
          ('🥕', 3),
          ('🥕', 2),
          ('🍆', 1),
          ('🍅', 4),
          ('🍅', 5),
          ('🍅', 3),
      ])
      | 'Saturated sum' >> beam.CombinePerKey(saturated_sum)
      | beam.Map(print))

('🥕', 5)
('🍆', 1)
('🍅', 8)


## **CombineValues:**


*   Combines an iterable of values in a keyed collection of elements.

*   CombineValues accepts a function that takes an iterable of elements as an input, and combines them to return a single element. 

*   CombineValues expects a keyed PCollection of elements, where the value is an iterable of elements to be combined.


In [8]:
with beam.Pipeline() as pipeline:
  total = (
      pipeline
      | 'Create produce counts' >> beam.Create([
          ('🥕', [3, 2]),
          ('🍆', [1]),
          ('🍅', [4, 5, 3]),
      ])
      | 'Sum' >> beam.CombineValues(sum)
      | beam.Map(print))

('🥕', 5)
('🍆', 1)
('🍅', 12)
